In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

classes, counts = np.unique(y, return_counts=True)
total = counts.sum()
percentages = (counts / total) * 100

# Imprimir resultados
print(f"Clase 0: {percentages[0]:.1f}% ({counts[0]} muestras)")
print(f"Clase 1: {percentages[1]:.1f}% ({counts[1]} muestras)")

Clase 0: 90.0% (900 muestras)
Clase 1: 10.0% (100 muestras)


In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
smt = SMOTE(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

classes, counts = np.unique(y_train_res, return_counts=True)
total = counts.sum()
percentages = (counts / total) * 100

# Imprimir resultados
print(f"Clase 0: {percentages[0]:.1f}% ({counts[0]} muestras)")
print(f"Clase 1: {percentages[1]:.1f}% ({counts[1]} muestras)")

Clase 0: 50.0% (630 muestras)
Clase 1: 50.0% (630 muestras)


Vamos a entrenar 4 modelos diferentes:
1. Regresión Logística
2. Random Forest
3. XGBoost
4. XGBoost con SMOTE

In [5]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test),
        {"C": 1, "solver": "liblinear"}
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test),
        {"n_estimators": 30, "max_depth": 3}
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test),
        {"use_label_encoder": False, "eval_metric": "logloss"}
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test),
        {"use_label_encoder": False, "eval_metric": "logloss", "resampling": "SMOTE"}
    )
]

In [6]:
reports = []

for model_name, model, train_set, test_set, params in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    print(model_name, "\n", report)
    reports.append(report)

Logistic Regression 
 {'0': {'precision': 0.9454545454545454, 'recall': 0.9629629629629629, 'f1-score': 0.9541284403669725, 'support': 270.0}, '1': {'precision': 0.6, 'recall': 0.5, 'f1-score': 0.5454545454545454, 'support': 30.0}, 'accuracy': 0.9166666666666666, 'macro avg': {'precision': 0.7727272727272727, 'recall': 0.7314814814814814, 'f1-score': 0.749791492910759, 'support': 300.0}, 'weighted avg': {'precision': 0.9109090909090909, 'recall': 0.9166666666666666, 'f1-score': 0.91326105087573, 'support': 300.0}}
Random Forest 
 {'0': {'precision': 0.9607142857142857, 'recall': 0.9962962962962963, 'f1-score': 0.9781818181818182, 'support': 270.0}, '1': {'precision': 0.95, 'recall': 0.6333333333333333, 'f1-score': 0.76, 'support': 30.0}, 'accuracy': 0.96, 'macro avg': {'precision': 0.9553571428571428, 'recall': 0.8148148148148149, 'f1-score': 0.8690909090909091, 'support': 300.0}, 'weighted avg': {'precision': 0.9596428571428572, 'recall': 0.96, 'f1-score': 0.9563636363636364, 'support

In [7]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.9607142857142857,
   'recall': 0.9962962962962963,
   'f1-score': 0.9781818181818182,
   'support': 270.0},
  '1': {'precision': 0.95,
   'recall': 0.6333333333333333,
   'f1-score': 0.76,
   'support': 30.0},
  'accuracy': 0.96,
  'macro avg': {'precision': 0.9553571428571428,
   'recall': 0.8148148148148149,
   'f1-score': 0.8690909090909091,
   'support': 300.0},
  'weighted avg': {'precision': 0.9596428571428572,
   'recall':

➡️ Vamos a registrar las métricas y modelos en MLflow

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.models.signature import infer_signature

In [9]:
params

{'use_label_encoder': False, 'eval_metric': 'logloss', 'resampling': 'SMOTE'}

In [11]:
mlflow.set_experiment("Second Experiment")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    X_train, y_train = element[2]
    X_test, y_test = element[3]
    params = element[4]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):   
        # Log de hiperparámetros y nombre del modelo     
        mlflow.log_param("model", model_name)
        mlflow.log_params(params)
        
        # Log de todas las métricas
        for label, metrics in report.items():
            if isinstance(metrics, dict):
                for metric_name, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric_name}", value)
            else:
                mlflow.log_metric(label, metrics)
                
        # Log del dataset
        X_train_df = pd.DataFrame(X_train, columns=[f"feature_{i}" for i in range(X_train.shape[1])])
        X_train_df["label"] = y_train
        dataset_path = f"data/train_dataset_{model_name.replace(' ', '_')}.csv"
        X_train_df.to_csv(dataset_path, index=False)
        mlflow.log_artifact(dataset_path, artifact_path="datasets")
        mlflow.set_tag("dataset", "with_smote") if "SMOTE" in model_name else mlflow.set_tag("dataset", "original")

        # Log del modelo
        input_example = pd.DataFrame(X_test[:2], columns=[f"feature_{i}" for i in range(X.shape[1])])    
        signature = infer_signature(X_test, y_pred)
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, 
                                     "model", 
                                     input_example = input_example, 
                                     signature = signature)
        else:
            mlflow.sklearn.log_model(model, 
                                     "model",
                                     input_example = input_example, 
                                     signature = signature)  

🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/931683556860607386/runs/e6ffcc7970954c4585c25be2dc14377d
🧪 View experiment at: http://localhost:5000/#/experiments/931683556860607386
🏃 View run Random Forest at: http://localhost:5000/#/experiments/931683556860607386/runs/0021020fd45b4989a762d6e7b9a5de41
🧪 View experiment at: http://localhost:5000/#/experiments/931683556860607386
🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/931683556860607386/runs/61e3f973294744df97be7c1a172883f6
🧪 View experiment at: http://localhost:5000/#/experiments/931683556860607386
🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/931683556860607386/runs/f4762dd8fc88437c86d3d5f1b0ced465
🧪 View experiment at: http://localhost:5000/#/experiments/931683556860607386


In [ ]:
# rm -rf mlruns/.trash/*

✅¡Listo! Tu experimento y los 4 runs se registraron correctamente en MLflow 🎉

Podemos comparar la performance de nuestros modelos desde la UI:

<div align="center">
    <img src="imgs/mlflow_2.png" alt="Compare Runs MLflow" width="900"/>
</div>